In [2]:
#!pip install opencv-python
import cv2

import time
import numpy as np
from ipywidgets import widgets
from IPython.display import display, clear_output

import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import linregress

umbral_contorno = 500

In [3]:
def obtener_metadatos(video_path):
    # Abre el archivo de video
    cap = cv2.VideoCapture(video_path)

    # Obtiene el número total de frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Obtiene el FPS (frames por segundo)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Obtiene las dimensiones del frame
    ancho = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    alto = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    tamaño_frame = (ancho, alto)
    duracion = total_frames / fps

    # Cierra el video
    cap.release()

    return total_frames, fps, tamaño_frame,duracion

In [4]:
video_path = "perseida1.mp4"
total_frames, fps, tamaño_frame,duracion = obtener_metadatos(video_path)
print(f"Total de frames: {total_frames}")
print(f"FPS: {fps}")
print(f"Dimensiones del frame: {tamaño_frame}")
print(f"Duración del video: {duracion}")


Total de frames: 71
FPS: 24.781849912739965
Dimensiones del frame: (1392, 1088)
Duración del video: 2.865


In [5]:
#!pip install ipywidgets

Definición de funciones para la detección de video

In [6]:
# Función que superpone las imagenes de un array de imágenes

from ipywidgets import widgets, HBox, VBox
from IPython.display import display, clear_output
from PIL import Image

def superimpose_images(frames):
    
    # np.maximum.reduce para obtener el valor máximo de píxeles entre todas las imágenes
    resulting_image = np.maximum.reduce(frames)
    return resulting_image

In [7]:
# Función para mostrar la detección de movimiento en el notebook

def mostrar_detección(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()

    output_widget = widgets.Output()
    display(output_widget)
    
    total_frames, fps, tamaño_frame,duracion = obtener_metadatos(video_path)
    
    # inicialización de variables
    
    (x,y,w,h)=(0,0,0,0)
    rectangulos=[]
    frames = []
    frames_deteccion = []
    flag = False
    last_frame = frame2

    while cap.isOpened():
        diff = cv2.absdiff(frame1, frame2)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, None, iterations=5)
        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            if cv2.contourArea(contour) < umbral_contorno:
                continue
                
            rectangulos.append([x,y,w,h,cv2.contourArea(contour),w/h,h/w])
            cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 1)
            frames.append(frame1)
            
            with output_widget:
                clear_output(wait=True)
                display(widgets.Image(value=cv2.imencode('.png', frame1)[1].tobytes()))

        frame1 = frame2
        ret, frame2 = cap.read()
       
        # Wait 5 seconds
       
        if not ret:
            break
    
    cap.release()
    
    if len(rectangulos) < 5:
        resulting_image = last_frame
    
    else:
            
        # Obteniendo los índices para el patrón en el array principal
        primero = 0
        ultimo = len(rectangulos) - 1
        medio = ultimo // 2
        primer_intermedio = primero + (medio - primero) // 2
        segundo_intermedio = medio + (ultimo - medio) // 2
    
        # Índices en el array principal
        indices = [primero, primer_intermedio, medio, segundo_intermedio, ultimo]
            
        # Recorrer los subarrays en las posiciones definidas y pinta los rectangulos de la detección y la linea de la trayectoria
    
        x1 = rectangulos [0][0] + int(rectangulos [0][2]/2)
        y1 = rectangulos [0][1] + int(rectangulos [0][3]/2)
        
        for i in indices:
        
            cv2.rectangle(frames[i], (rectangulos [i][0], rectangulos [i][1]), (rectangulos [i][0] + rectangulos [i][2], rectangulos [i][1] + rectangulos [i][3]), (0, 255, 0), 1)
             
            x2 = rectangulos [i][0] + int((rectangulos [i][2])/2)
            y2 = rectangulos [i][1] + int((rectangulos [i][3])/2)
                       
            cv2.line(frames[i], (x1 , y1), (x2 , y2), (0, 255, 0), 1)
     
            frames_deteccion.append(frames[i])
                                           
            x1 = x2
            y1 = y2
        
        resulting_image = superimpose_images(frames_deteccion)
    
    # Escribir metadatos video
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (50, 50) # posición de inicio del texto
    fontScale = 1 # tamaño de la fuente
    color = (0, 255, 0) # color del texto en BGR
    thickness = 2 # grosor de la línea
    
    lines = [
    f" Video: {video_path}",
    f" Total frames: {total_frames}",
    f" Frames detectados:{len(rectangulos)}",
    f" Tiempo detectados (seg): {duracion}"]

    # Parámetros para cv2.putText
    font = cv2.FONT_HERSHEY_SIMPLEX
    x_start = 10  # posición horizontal inicial
    y_start = 50  # posición vertical inicial
    y_offset = 30  # espacio entre líneas
    fontScale = 0.7  # tamaño de la fuente
    color = (0, 255, 0)  # color del texto en BGR
    thickness = 1  # grosor de la línea

    for line in lines:
        cv2.putText(resulting_image, line, (x_start, y_start), font, fontScale, color, thickness, cv2.LINE_AA)
        y_start += y_offset

    # Widget de visualización
    
    out = widgets.Output(layout={'border': '1px solid black'})
    with out:
        display(Image.fromarray(resulting_image))

    display(out)
    
    cv2.imwrite("deteccion.png",resulting_image)
                        
    return {'fps':fps,'rectangulos':rectangulos,'image':resulting_image}

In [8]:
# Ejemplo de llamada a la funcion 

video_path = "./ave.mp4"
mostrar_detección(video_path)

Output()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

{'fps': 24.778331106522533,
 'rectangulos': [[1150, 177, 25, 26, 512.0, 0.9615384615384616, 1.04],
  [1140, 172, 26, 27, 548.0, 0.9629629629629629, 1.0384615384615385],
  [1130, 169, 23, 27, 514.0, 0.8518518518518519, 1.173913043478261],
  [1126, 167, 24, 28, 544.5, 0.8571428571428571, 1.1666666666666667],
  [1118, 165, 24, 29, 561.0, 0.8275862068965517, 1.2083333333333333],
  [1114, 165, 26, 28, 622.5, 0.9285714285714286, 1.0769230769230769],
  [1100, 163, 29, 31, 703.5, 0.9354838709677419, 1.0689655172413792],
  [1102, 163, 22, 30, 520.0, 0.7333333333333333, 1.3636363636363635],
  [1091, 166, 24, 29, 571.5, 0.8275862068965517, 1.2083333333333333],
  [1085, 166, 30, 32, 777.0, 0.9375, 1.0666666666666667],
  [1083, 166, 27, 32, 681.0, 0.84375, 1.1851851851851851],
  [1082, 169, 24, 32, 577.0, 0.75, 1.3333333333333333],
  [1081, 169, 25, 30, 586.5, 0.8333333333333334, 1.2],
  [1079, 168, 24, 31, 605.5, 0.7741935483870968, 1.2916666666666667],
  [1076, 172, 24, 28, 567.5, 0.8571428571428

In [9]:
# Función para caracterizar el movimiento
# detecta objeto y define el rectángulo del contorno
#
# con el array de rectángulos de la deteccion se extrae:
#
# x,y,w,h: define el rectangulo como coordenadas (x,y) iniciales w:ancho h:alto
# área del contorno (pixeles^2)
# w/h: ratio ancho/alto
# h/w: ratio alto/ancho

def extraccion_deteccion (video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()

    output_widget = widgets.Output()
    display(output_widget)
    
    total_frames, fps, tamaño_frame,duracion = obtener_metadatos(video_path)
    
        
    # inicialización de variables
    
    (x,y,w,h)=(0,0,0,0)
    rectangulos=[]
        
        
    while cap.isOpened():
        diff = cv2.absdiff(frame1, frame2)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, None, iterations=5)
        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            
            #si el contorno es menor del umbral 
            
            if cv2.contourArea(contour) < umbral_contorno:
                continue
            
            rectangulos.append([x,y,w,h,cv2.contourArea(contour),w/h,h/w])
            cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 1)
                                         
        frame1 = frame2
        ret, frame2 = cap.read()
                      
        if not ret:
            break

    cap.release()
   
    return {'fps':fps,'rectangulos':rectangulos}


In [10]:
# Función para extracción de variables
#
# con el array de rectángulos de la deteccion se extrae: (funcion independiente)
#
# - Tiempo del objeto detectado en segundos
# - Variabilidad (cociente entre el máximo y mínimo tamaño de los contornos)
# - Si tiene o no estela:ratio de h/v
# - R2 (ajuste lineal)

def extraccion_variables(p_deteccion):

    deteccion=p_deteccion['rectangulos']

    frames_deteccion = len([sub_arr[4] for sub_arr in deteccion if len(sub_arr) > 4 and sub_arr[4] > umbral_contorno])
    
    if frames_deteccion <5:
        return {'num_frames_deteccion':frames_deteccion,'variabilidad':0,'max_contorno':0,'min_contorno':0,
                'estelaH':0,'estelaV':0,'estela':0,'R2':0}
    
    
    max_contorno = max([sub_arr[4] for sub_arr in deteccion if len(sub_arr) > 4 and sub_arr[4] > umbral_contorno])
    min_contorno = min([sub_arr[4] for sub_arr in deteccion if len(sub_arr) > 4 and sub_arr[4] > umbral_contorno])
    variabilidad = max_contorno / min_contorno
    
    ratio_estelaH = max([sub_arr[3]/sub_arr[2] for sub_arr in deteccion if len(sub_arr) > 4 
                         and sub_arr[4] > umbral_contorno])
    ratio_estelaV = max([sub_arr[2]/sub_arr[3] for sub_arr in deteccion if len(sub_arr) > 4 
                         and sub_arr[4] > umbral_contorno])
    ratio_estela = max (ratio_estelaH,ratio_estelaV)
    
    selected_points = [deteccion[0], deteccion[len(deteccion)//4], deteccion[len(deteccion)//2], 
                       deteccion[3*len(deteccion)//4], deteccion[-1]]

    # Separar las coordenadas en dos listas (x e y)
    x = [vector[0] + (vector[2]/2) for vector in selected_points]
    y = [vector[1] + (vector[3]/2) for vector in selected_points]
    
        
    # Ajuste lineal usando scipy
    
    try: 
        slope, intercept, r_value, p_value, std_err = linregress(x, y)

    except ValueError as e:
        
    # Comprueba si el mensaje de error
    
        if str(e) == "Cannot calculate a linear regression if all x values are identical":
            print("Error: Todos los valores de x son idénticos. No se puede calcular la regresión lineal.")
            return {'num_frames_deteccion':frames_deteccion,'variabilidad':variabilidad,'max_contorno':max_contorno,
                    'min_contorno':min_contorno,'estelaH':ratio_estelaH,'estelaV':ratio_estelaV,'estela':ratio_estela,
                    'R2':0}
    
        else:
            raise
        
    return {'num_frames_deteccion':frames_deteccion,'variabilidad':variabilidad,'max_contorno':max_contorno,
           'min_contorno':min_contorno,'estelaH':ratio_estelaH,'estelaV':ratio_estelaV,'estela':ratio_estela,
            'R2':r_value**2}

Para extraer las variables de un conjunto de videos, la función extraccion_variables_lote devuelve un dataframe con el resultado de la extracción de los videos de un directorio dado.

In [11]:
import os
import pandas as pd

def extraccion_variables_lote(path,clase,fichero_variables):

# Extrae variables para un directorio que contiene videos mp4
# Dado un path, clase (1: bolido y 0: no bolido), fichero_variables donde se escribiran csv para entrenamiento
# 
# Escribe fichero_variables como csv con: 
#      Nombre de fichero mp4
#      Clase (1 o 0)
#      Tiempo de detección en segundos
#      Array con variables:
#      Variabilidad (tamaño de rectángulo)
#      Si tiene o no estela:ratio de h/v
#      R2 ajuste trayectoria lineal

    # Comprobar si el directorio existe
    
    if not os.path.exists(path):
        print("El directorio especificado no existe.")
        return
    
    # Lista para almacenar los datos antes de convertirlos a DataFrame
    data = []
    
    # Recorrer los ficheros en el directorio
    for filename in os.listdir(path):
        print(filename)
        if filename.endswith(".mp4"):
            
            total_frames, fps, tamaño_frame,duracion = obtener_metadatos(os.path.join(path, filename))
            
            # Llamar a tu función extraccion_variables_lote
            variables = extraccion_variables(extraccion_deteccion(os.path.join(path, filename)))
            
            # Añadir al dataset
            data.append({
                "Nombre_Fichero": filename,
                "Clase": clase,
                "Tiempo_Deteccion": variables['num_frames_deteccion']/fps,
                "Array_Resultado": variables
            })

    # Convertir la lista de datos a DataFrame
    df = pd.DataFrame(data)
    
    # Guardar el DataFrame en un archivo CSV
    df.to_csv(fichero_variables, index=False)

    print("El procesamiento ha finalizado y los resultados se han guardado en",path)
    
    return df

In [12]:
# Ejemplo de extracción de variables para el conjunto de datos de entrenamiento clase 1: Bólidos
# Descomentar para ejecutar (el proceso tarda unos 20 minutos)
 
# extraccion_variables_lote ("./meteor",1,"./clase1.csv")

In [13]:
# Ejemplo de extracción de variables para el conjunto de datos de entrenamiento clase 0: No Bólidos
# Descomentar para ejecutar (el proceso tarda unos 20 minutos)

#extraccion_variables_lote ("./non_meteor",0,"./clase0.csv")

In [14]:
# crea dataset para entrenamiento desde un fichero csv: file

import ast
def crear_dataset (file):

    bolidos = pd.read_csv(file, sep = ',') # carga del CSV, usando como separador , 'comas'
    
    bolidos['Array_Resultado'] = bolidos['Array_Resultado'].apply(ast.literal_eval)

    bolidos = bolidos.join(pd.json_normalize(bolidos['Array_Resultado'])).drop(columns=['Array_Resultado'])
    
    #bolidos['R2'] = bolidos['R2'].astype(float)
    
    bolidos['R2'] = pd.to_numeric(bolidos['R2'], errors='coerce')

    return bolidos

# Codigo Entrenamiento


In [15]:
# Codigo Entrenamiento

# carga inicial dataframe df_bolidos con ammbas clases (0 y 1  para negativos y para positivos)

df_bolidos = pd.concat([crear_dataset("./clase1.csv"),crear_dataset("./clase0.csv")])
df_bolidos = df_bolidos.sample(frac=1).reset_index(drop=True)

# Se limpia el dataframe df_bolidos antes del seguir procesando:
# - eliminando los de clase 0 y 1 con 0 frames_deteccion: el algoritmo de deteccion ha fallado
# - eliminamos los de clase 0 y 1 con 'NA' por fallo en la regresión lineal de trayectoria

df_filtrado = df_bolidos[df_bolidos['num_frames_deteccion'] > 0]
df_filtrado = df_filtrado[df_filtrado['R2'] > 0]


# Se separan en df_X y df_Y para los dataset de entrenamiento

df_X = df_filtrado [['Tiempo_Deteccion','num_frames_deteccion','variabilidad','max_contorno','min_contorno','estelaH','estelaV','estela','R2']]
df_Y = df_filtrado [['Clase']]

#df_X = df_bolidos [['Tiempo_Deteccion','num_frames_deteccion','variabilidad','max_contorno','min_contorno','estelaH','estelaV','estela','R2']]
#df_Y = df_bolidos [['Clase']]


# Separamos en train/test
from sklearn.model_selection import train_test_split

# Utilizamos una proporción 70/30. El parámetro random_state lo utilizamos para obtener los mismos resultados siempre
X_bolidos_train, X_bolidos_test, y_bolidos_train, y_bolidos_test = train_test_split(df_X, df_Y,test_size = 0.3, random_state = 42)

print("Dimensiones de conjuntos de entrenamiento: variables de entrada (X) y salida (y)", X_bolidos_train.shape, y_bolidos_train.shape)
print("Dimensiones de conjuntos de validación: variables de entrada (X) y salida (y)",X_bolidos_test.shape, y_bolidos_test.shape)


Dimensiones de conjuntos de entrenamiento: variables de entrada (X) y salida (y) (6865, 9) (6865, 1)
Dimensiones de conjuntos de validación: variables de entrada (X) y salida (y) (2943, 9) (2943, 1)


Entrenamiento de módelos
Se van a entrenar varios modelos de Clasificación:

* Regresión Logística
* Árbol de Decisión
* KNN
* SVC
* Redes Neuronales

Y se obtendrán las métricas de recall, precisión y matriz de confusión para analizar su rendimiento

# Regresión Logística

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Creamos el objeto del modelo,fijando la semilla para evitar aleatoriedad. Se sube el parametro max_iter para que finalice
logreg = LogisticRegression(random_state=42,max_iter=10000)

# Entrenamos con el conjunto de train y su target
X_bolidos_train = X_bolidos_train.select_dtypes(include = 'number')
X_bolidos_test = X_bolidos_test.select_dtypes(include = 'number')

logreg.fit(X_bolidos_train, y_bolidos_train[['Clase']].values.ravel())
logreg.coef_.shape

print('Accuracy en train: ', logreg.score(X_bolidos_train, y_bolidos_train[['Clase']]))
print('Accuracy en test: ', logreg.score(X_bolidos_test, y_bolidos_test[['Clase']]))

# Calculamos las métricas globales, con el parámetro average

preds_proy = logreg.predict(X_bolidos_test)
probs_proy = logreg.predict_proba(X_bolidos_test)
precision = precision_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
recall = recall_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
cm = confusion_matrix(y_bolidos_test[['Clase']], preds_proy)

print('Precision: %.3f' % precision)
print('Recall %.3f' % recall)
print(cm)

Accuracy en train:  0.9367807720320466
Accuracy en test:  0.9367991845056065
Precision: 0.937
Recall 0.937
[[1775  158]
 [  28  982]]


# Arból de Decisión

In [17]:
from sklearn.tree import DecisionTreeClassifier

# Inicialiamos con sus parámetros por defecto (salvo la semilla)
tree_class = DecisionTreeClassifier(random_state = 42)

# Entrenamiento
tree_class.fit(X_bolidos_train, y_bolidos_train[['Clase']])
print('Accuracy en train: ',tree_class.score(X_bolidos_train, y_bolidos_train[['Clase']]))
print('Accuracy en test: ', tree_class.score(X_bolidos_test, y_bolidos_test[['Clase']]))

# Calculamos las métricas globales, con el parámetro average

preds_proy = tree_class.predict(X_bolidos_test)
probs_proy = tree_class.predict_proba(X_bolidos_test)
precision = precision_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
recall = recall_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
cm = confusion_matrix(y_bolidos_test[['Clase']], preds_proy)

print('Precision: %.3f' % precision)
print('Recall %.3f' % recall)
print(cm)

Accuracy en train:  1.0
Accuracy en test:  0.9364593951749915
Precision: 0.936
Recall 0.936
[[1834   99]
 [  88  922]]


# KNN

In [18]:
from sklearn.neighbors import KNeighborsClassifier

# Por defecto, k = 5
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_bolidos_train, y_bolidos_train[['Clase']].values.ravel())
print('Accuracy en train: ',knn.score(X_bolidos_train, y_bolidos_train[['Clase']]))
print('Accuracy en test: ',knn.score(X_bolidos_test, y_bolidos_test[['Clase']]))

preds_proy = knn.predict(X_bolidos_test)
probs_proy = knn.predict_proba(X_bolidos_test)
precision = precision_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
recall = recall_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
cm = confusion_matrix(y_bolidos_test[['Clase']], preds_proy)

print('Precision: %.3f' % precision)
print('Recall %.3f' % recall)
print(cm)


Accuracy en train:  0.9238164603058995
Accuracy en test:  0.8854909955827387
Precision: 0.885
Recall 0.885
[[1723  210]
 [ 127  883]]


# SVC

In [19]:
from sklearn.svm import SVC

svc = SVC(random_state=42,probability=True) # parámetros por defecto

svc.fit(X_bolidos_train, y_bolidos_train[['Clase']].values.ravel())
print('Accuracy en train: ',svc.score(X_bolidos_train, y_bolidos_train[['Clase']]))
print('Accuracy en test: ',svc.score(X_bolidos_test, y_bolidos_test[['Clase']]))

preds_proy = svc.predict(X_bolidos_test)
probs_proy = svc.predict_proba(X_bolidos_test)
precision = precision_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
recall = recall_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
cm = confusion_matrix(y_bolidos_test[['Clase']], preds_proy)

print('Precision: %.3f' % precision)
print('Recall %.3f' % recall)
print(cm)

Accuracy en train:  0.8569555717407138
Accuracy en test:  0.8501529051987767
Precision: 0.850
Recall 0.850
[[1518  415]
 [  26  984]]


# MLP

In [20]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state = 42,max_iter=1500)

mlp.fit(X_bolidos_train, y_bolidos_train[['Clase']].values.ravel())

print('Accuracy en train: ',mlp.score(X_bolidos_train, y_bolidos_train[['Clase']]))
print('Accuracy en test: ',mlp.score(X_bolidos_test, y_bolidos_test[['Clase']]))

preds_proy = mlp.predict(X_bolidos_test)
probs_proy = mlp.predict_proba(X_bolidos_test)
precision = precision_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
recall = recall_score(y_bolidos_test[['Clase']], preds_proy, average = 'micro')
cm = confusion_matrix(y_bolidos_test[['Clase']], preds_proy)

print('Precision: %.3f' % precision)
print('Recall %.3f' % recall)
print(cm)

Accuracy en train:  0.9159504734158777
Accuracy en test:  0.9119945633707102
Precision: 0.912
Recall 0.912
[[1704  229]
 [  30  980]]


# Combinando modelos

In [21]:
from sklearn.ensemble import VotingClassifier

# Combinamos los 3 modelos anteriores (excluyendo a Vector Soporte por ser el peor)
vc = VotingClassifier(estimators = [('mlp',mlp),('logreg',logreg),('tree_class',tree_class),('svc',svc),('knn',knn)])
                      
# ojo con train no con test

vc.fit(X_bolidos_train, y_bolidos_train.values.ravel())

print('Accuracy modelo combinado: %.3f' % vc.score(X_bolidos_test, y_bolidos_test))

Accuracy modelo combinado: 0.922


# Optimización de Hiperparámetros

In [245]:
from sklearn.model_selection import GridSearchCV

# Parámetros de búsqueda y posibles valores
grid_parameters = {'kernel': ['linear', 'rbf', 'poly'], 'C': [10.0, 1.0, 0.1, 0.01, 0.001]}

# Grid search
grid_SVC = GridSearchCV(SVC(random_state=42), param_grid = grid_parameters, cv = 3, scoring = 'accuracy')

# Comenzamos el entrenamiento
grid_SVC.fit(X_bolidos_train, y_bolidos_train.values.ravel())

print(grid_SVC.best_params_)

print(grid_SVC.best_score_)

print(grid_SVC.best_estimator_)


{'C': 10.0, 'kernel': 'linear'}
0.9395479647773226
SVC(C=10.0, kernel='linear', random_state=42)


In [244]:
from sklearn.model_selection import GridSearchCV

# Parámetros de búsqueda y posibles valores
grid_parameters = {'hidden_layer_sizes': [(20,), (30,), (15, 15)], 
                   'activation': ['tanh', 'relu'] }

# Grid search
grid_MLP = GridSearchCV(MLPClassifier(random_state=42,max_iter=1500), param_grid = grid_parameters, cv = 3, scoring = 'accuracy')

# Comenzamos el entrenamiento
grid_MLP.fit(X_bolidos_train, y_bolidos_train.values.ravel())

print(grid_MLP.best_params_)

print(grid_MLP.best_score_)

print(grid_MLP.best_estimator_)

{'activation': 'tanh', 'hidden_layer_sizes': (15, 15)}
0.9373639230290607
MLPClassifier(activation='tanh', hidden_layer_sizes=(15, 15), max_iter=1500,
              random_state=42)


In [243]:
from sklearn.model_selection import GridSearchCV

# Parámetros de búsqueda y posibles valores
grid_parameters = {'criterion': ['gini', 'entropy', 'log_loss'], 'splitter': ['best', 'random'],'max_depth' : [1,2,3,4,5,6,7,8] }

# Grid search
grid_Tree = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid = grid_parameters, cv = 3, scoring = 'accuracy')

# Comenzamos el entrenamiento
grid_Tree.fit(X_bolidos_train, y_bolidos_train.values.ravel())

print(grid_Tree.best_params_)

print(grid_Tree.best_score_)

print(grid_Tree.best_estimator_)

{'criterion': 'gini', 'max_depth': 6, 'splitter': 'best'}
0.9418785215803566
DecisionTreeClassifier(max_depth=6, random_state=42)


In [242]:
from sklearn.model_selection import GridSearchCV

# Parámetros de búsqueda y posibles valores
k_range = list(range(1, 31))
grid_parameters = dict(n_neighbors=k_range)

# Grid search
grid_KNN = GridSearchCV(KNeighborsClassifier(),param_grid = grid_parameters, cv = 10, scoring = 'accuracy')

# Comenzamos el entrenamiento
grid_KNN.fit(X_bolidos_train, y_bolidos_train.values.ravel())

print(grid_KNN.best_params_)

print(grid_KNN.best_score_)

print(grid_KNN.best_estimator_)

{'n_neighbors': 19}
0.9002187649857198
KNeighborsClassifier(n_neighbors=19)


In [ ]:
c = VotingClassifier(estimators = [('mlp',grid_MLP),('logreg',logreg),('tree_class',grid_Tree),('svc',grid_SVC),('knn',grid_KNN)])
                      
# ojo con train no con test

vc.fit(X_bolidos_train, y_bolidos_train.values.ravel())

print('Accuracy modelo combinado: %.3f' % vc.score(X_bolidos_test, y_bolidos_test))

# Guardamos modelo

In [24]:
# Se almacena el modelo vc
from joblib import dump

dump(vc, 'modelo_combinado_final.joblib')

['modelo_combinado_final.joblib']

Evaluación de Vídeo

In [23]:
# Para un video mp4 se ejecuta el pipeline de detección y clasificación

video_path="perseida1.mp4"

mostrar_detección(video_path)

total_frames, fps, tamaño_frame,duracion = obtener_metadatos(video_path)
D = extraccion_deteccion(video_path)
X = extraccion_variables(D)

df = pd.DataFrame(X,index=[0])

df.insert(0, 'Tiempo_Deteccion', df['num_frames_deteccion'][0] / fps)

clasificacion = vc.predict(df)

print("La clasificacion es:",clasificacion[0])

imagen = cv2.imread('deteccion.png', cv2.IMREAD_UNCHANGED)

 
# Parámetros para cv2.putText
font = cv2.FONT_HERSHEY_SIMPLEX
x_start = 10  # posición horizontal inicial
y_start = 200  # posición vertical inicial
y_offset = 30  # espacio entre líneas
fontScale = 0.7  # tamaño de la fuente
color = (0, 255, 0)  # color del texto en BGR
texto = "Bolido [1]"
thickness = 2  # grosor de la línea

if clasificacion[0] == 0:
    color = (0,0,255)
    texto = "No bolido [0]"
    
cv2.putText(imagen,texto,(10,200), font, fontScale, color, thickness, cv2.LINE_AA)

display(widgets.Image(value=cv2.imencode('.png', imagen)[1].tobytes()))
  


Output()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Output()

La clasificacion es: 1


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05p\x00\x00\x04@\x08\x02\x00\x00\x00\xc8\xe0\xec\xe…